# 2.) ADS Project - SwissSuperLeague

Verwendete Quellen:
- Hands On ML with Scikit-Learn, Keras und Tensorflow, A.Geron, O'Reilly - gemäss Vorlesung
- Decision Tree Regression in 6 Steps with Python: https://medium.com/pursuitnotes/decision-tree-regression-in-6-steps-with-python-1a1c5aa2ee16
- Day 19 - Regression trees: http://alumni.media.mit.edu/~tpminka/courses/36-350.2001/lectures/day19/
- Scrape FBref data: https://github.com/parth1902/Scrape-FBref-data/blob/master/README.md
- StatsBomb Release Free 2020/21 FA Women’s Super League Data & Updated ‘R’ Guide: https://statsbomb.com/articles/soccer/statsbomb-release-free-2020-21-fa-womens-super-league-data-updated-r-guide/
- Udacity Capstone Project: https://github.com/Matheuskempa/My_Udacity_Capstone
- Code aus Vorlesungswoche 8
- Codes aus Woche 7: Exploratory data analysis and vizualizations von DSF

## Data Cleansing

Zuerst importieren wir die relevanten libraries.

In [ ]:
import pandas as pd
import numpy as np

Einlesen der von zuvor gescrapten Daten aus csv. file

In [ ]:
df = pd.read_csv('Swiss.csv')

Daten anschauen

In [ ]:
df.head()

Anzahl Daten-Zeilen pro Spalte sowie Datentypen anschauen

In [ ]:
df.info()

Zwischenschritt: Spalte 'Score' trennen in zwei

In [ ]:
# Für Score Spalte '-' entfernen
df["Score"] = df["Score"].replace("\–", "", regex=True)

# Resultat in Heim und Auswärts Spalte trennen
df['HomeScore'] = df['Score'].str[0:1]
df['HomeScore'] = df['HomeScore'].astype(str).astype(float)
df['AwayScore'] = df['Score'].str[1:2].astype(str)
df['AwayScore'] = df['AwayScore'].astype(str).astype(float)

Daten anschauen mit getrennten Spalten

In [ ]:
df.head()

Zwischenschritt 2: Neue 'Ergebnis' Spalte schaffen mittels der getrennten Score (HomeScore/AwayScore)

In [ ]:
#Neue Spalte mit Niederlage(0)/Sieg(1)/Unentschieden(2)
df['Ergebnis'] = "2"
df['Ergebnis'] = df['Ergebnis'].astype(str).astype(float)

# Unterdrücken von SettingWithCopyWarning
pd.options.mode.chained_assignment = None

#for Schleife unm die spalte "Ergebnis" zu befüllen
for index, row in df.iterrows():
    home = row['HomeScore']
    away = row['AwayScore']
    if home < away:
        df["Ergebnis"][index]  = 0
    elif home > away:
        df["Ergebnis"][index] = 1
    elif ((home-away) == 0):
        df["Ergebnis"][index]  = 2

Zwischenschritt 2: Überprüfen ob 'Ergebnis' Spalte sichtbar

In [ ]:
df.head(10)

In [ ]:
df.info()

In [ ]:
df.count()

Checken für duplicated columns -> page breaks in unserem table

In [ ]:
df.duplicated().sum()

Drop duplicates

In [ ]:
df = df.drop_duplicates()
df.duplicated().sum()

In [ ]:
df.count()

Checken für NA cells

In [ ]:
df.isna().sum()

Wir berücksichtigen nur Zeilen mit einem Score weil die anderen Zeilen NA oder keine gespielten Spiele sind und daher unwichtig sind für uns. 

In [ ]:
df = df[df['Score'].notna()]

In [ ]:
df.count()

Droppen von irrelevanten Spalten

In [ ]:
df=df.drop(columns=['Notes', 'Match Report', 'Referee'])

In [ ]:
df.count()

## EDA - Exploratory Data Visualization

Zuerst importieren wir die relevanten libraries.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

Hier visualiseren wir die nummerischen Stats um interessante Insgights zu bekommen

In [ ]:
df.describe()

Boxplot der Attendance Verteilung zeigt

In [ ]:
import seaborn as sns
plt.figure(figsize=(10,2))
plt.ticklabel_format(style='plain')
sns.boxplot(x=df['Attendance'], color="green")
print('Mean of Attendance: ' +str(int(np.mean(df['Attendance']))))

Histogramm der Häufigkeit der 'Attendance'

In [ ]:
fig = plt.figure( figsize=(8,4) )
plt.xticks(fontsize=12, rotation=0)
plt.yticks(fontsize=12, rotation=0)
n, bins, patches = plt.hist(x=df['Attendance'], bins=20, color='#3d85c6', alpha=1, rwidth=0.95)
plt.axvline(df['Attendance'].mean(), color='r', linestyle='dashed', linewidth=3)
plt.grid(True)

plt.title('Histogram of Attendance', fontsize=16, pad=10)
plt.ticklabel_format(style='plain')
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Attendance', fontsize=14, labelpad=10)
plt.ylabel('Frequency', fontsize=14, labelpad=10)

Spider chart um die durchschnittliche Besucherzahlen der verschiedenen Stadien zu vergleichen

In [ ]:
df_spider= df[['Venue','Attendance']]
df_spider_2 = df_spider.groupby(df_spider['Venue']).mean()
fig = px.line_polar(df_spider_2, r='Attendance', 
                    theta=df_spider_2.index,  
                    line_close=True
                   )
fig.update_layout(width=500,height=370)
fig.show()

## Machine Learning modelling

Die Daten sind nun gescrapet und visualisiert worden. Nun werden sie in einem ML Modell weiter verwertet.


Zuerst importieren wir die relevanten libraries.

In [ ]:
import requests
import warnings
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
warnings.filterwarnings("ignore")

Variablen welche nicht relevant sind für das ML Modell droppen wir von unserem Datensatz

In [ ]:
eingabe = df.drop(columns=['Wk','Score', 'Day', 'Date', 'Time','Home','Away','Venue','Ergebnis'])
eingabe.info()

Wir definieren zuerst die dependent und independent Variabeln

In [ ]:
x = pd.DataFrame(eingabe)
y = df["Ergebnis"]

Wir bereiten die Daten vor in dem wie sie in Test und Training Datensätze splitten

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=20)

In [ ]:
print(X_train)

### Random Forest

Jetzt executen wir RandomForestClassifier ML Algortihmus für unsere Trainingsdaten

In [ ]:
model_RF = RandomForestClassifier (n_estimators = 100, random_state = 30)
model_RF.fit (X_train, y_train)

Als nächsten machen wir eine Prediction für unser Modell

In [ ]:
prediction_test = model_RF.predict(X_test)

Wir zeigen die Accuracy

In [ ]:
print ("Accuracy = ", accuracy_score(y_test, prediction_test))

Jetzt zeigen wir die Verteilung der Wichtigkeit der einzelnen Features im Random Forest ausgeben

In [ ]:
feature_list = list(x.columns)
feature_imp = pd.Series(model_RF.feature_importances_, index=feature_list).sort_values(ascending=False)
#print (model.feature_importances_)
print (feature_imp)

Visualisierung der Wichtigkeit:

In [ ]:
importance = model_RF.feature_importances_

list_1 = []
list_2 = []
for i,j in zip(importance,eingabe.columns):
    list_1.append(str(j))
    list_2.append(i)
    
df_name = pd.DataFrame(list_1,columns=["Feature"])
df_number = pd.DataFrame(list_2,columns=["Wichtigkeit"])

df_ranking = pd.concat([df_name, df_number], axis=1)
df_final = df_ranking.sort_values("Wichtigkeit",ascending=False).reset_index(drop=True)
df_final.Column = df_final.Feature.astype(str)
f, ax = plt.subplots(figsize=(8, 4))
sns.barplot(x="Wichtigkeit", y="Feature", data=df_final)

### Neural Network

Jetzt trainieren wir unser Model noch mittels eines NeuralNetworks

In [ ]:
model_NN = tf.keras.models.Sequential()
model_NN.add(tf.keras.layers.Flatten())
model_NN.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model_NN.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model_NN.add(tf.keras.layers.Dense(3, activation=tf.nn.softmax))

model_NN.compile(optimizer='adam',
        loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model_NN.fit(X_train, y_train, epochs=30)

Hier die Accuracy zum Neural Network

In [ ]:
val_loss, val_acc = model_NN.evaluate (X_test, y_test)
print (val_loss, val_acc)